In [1]:
import os

# Check current working directory
print(os.getcwd())

# Change to the root directory of your project, if necessary
os.chdir('C:/Users/yashv/Downloads/HADLN_implementation/HADLN_implementation/')
print(os.getcwd())


c:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation\notebooks
C:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation


In [2]:
# Cell 1: Import Libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from models.resnet_A import build_resnet_with_attention
from preprocessing.resnet_preprocessing import load_data, split_data  # Assuming you have a preprocessing script named data_preprocessing.py
from datetime import datetime

c:\Users\yashv\.conda\envs\tf\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2000, 1)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 2000, 32)     544         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 2000, 32)    128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 2000, 32)     0           ['batch_normalization[0][0]']

In [3]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
import pandas as pd
# Cell 2: Load and Preprocess Data
# Load the DWT-transformed data
dwt_file_path = 'data/processed/dwt_features_ecg.csv'  # Update with the correct path
data = pd.read_csv(dwt_file_path)

# Extract features and labels
X = data.iloc[:, :-1].values  # All columns except the last one are features
y = data.iloc[:, -1].values  # Last column is the label

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# One-hot encode labels
num_classes = 4
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

# Reshape input data to match model expectations (batch_size, time_steps, num_features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [5]:
# Cell 3: Extract Time Step and Number of Features
time_step = X_train.shape[1]
num_features = X_train.shape[2]

In [6]:
# Cell 4: Build the Model

# Build and compile the ResNet with attention model
model = build_resnet_with_attention((time_step, num_features), num_classes)

In [11]:

# Cell 5: Compile the Model
# Define learning rate scheduler, TensorBoard, and ModelCheckpoint callbacks
# Define callbacks
log_dir = f"logs/tensorboard/ResNet_DWT/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
checkpoint_callback = ModelCheckpoint(filepath='resnet_dwt_best_model.h5', save_best_only=True, verbose=1)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks = [checkpoint_callback, tensorboard_callback, lr_scheduler, early_stopping]


# Calculate class weights to handle class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights = dict(enumerate(class_weights))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 2026, 1)]    0           []                               
                                                                                                  
 conv1d_48 (Conv1D)             (None, 2026, 32)     544         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_48 (BatchN  (None, 2026, 32)    128         ['conv1d_48[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_32 (Activation)     (None, 2026, 32)     0           ['batch_normalization_48[0]

In [13]:
# Cell 6: Train the Model

# Train the model with class weights and callbacks
history = model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_test, y_test_one_hot),
    epochs=100,
    batch_size=16,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 1/100
427/427 [==============================] - ETA: 0s - loss: 1.5928 - accuracy: 0.3996
Epoch 1: val_loss did not improve from 1.61422
427/427 [==============================] - 40s 92ms/step - loss: 1.5928 - accuracy: 0.3996 - val_loss: 1.6914 - val_accuracy: 0.0287 - lr: 1.2500e-04
Epoch 2/100
427/427 [==============================] - ETA: 0s - loss: 1.5596 - accuracy: 0.2932
Epoch 2: val_loss improved from 1.61422 to 1.59366, saving model to resnet_dwt_best_model.h5
427/427 [==============================] - 39s 92ms/step - loss: 1.5596 - accuracy: 0.2932 - val_loss: 1.5937 - val_accuracy: 0.2696 - lr: 1.2500e-04
Epoch 3/100
427/427 [==============================] - ETA: 0s - loss: 1.5382 - accuracy: 0.2476
Epoch 3: val_loss did not improve from 1.59366
427/427 [==============================] - 39s 91ms/step - loss: 1.5382 - accuracy: 0.2476 - val_loss: 1.7105 - val_accuracy: 0.0287 - lr: 1.2500e-04
Epoch 4/100
427/427 [==============================] - ETA: 0s - loss: 1

KeyboardInterrupt: 

In [ ]:
# Cell 7: Evaluate the Model on the Test Set
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# Cell 8: Plot Training History
# Plot accuracy and loss curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Accuracy
ax1.plot(history.history['accuracy'], label='Train Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.set_title('Training and Validation Accuracy')
ax1.legend()

# Loss
ax2.plot(history.history['loss'], label='Train Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Loss')
ax2.set_title('Training and Validation Loss')
ax2.legend()

plt.tight_layout()
plt.show()



In [ ]:
# Cell 9: Save the Model
# Save the trained model for future use
model.save('models/resnet_trained.h5')